# Paraglider Detection Model Training

This notebook trains a YOLOv8 model to detect paragliders in images. It covers:
1. Setting up dependencies and loading the dataset from Roboflow
2. Training the YOLOv8 model
3. Evaluating and visualizing results
4. Deploying to Roboflow
5. Optimizing for edge devices (QAI Hub)


In [ ]:
# Install required packages
# roboflow: For downloading the dataset
# ultralytics: For YOLOv8 training and inference
!pip install -q roboflow
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.8 MB/s eta 0:00:00


In [ ]:
# Setup: Load environment variables and initialize dependencies
import os
from dotenv import load_dotenv
from IPython import display
display.clear_output()

# Load environment variables from .env file
load_dotenv()

# Verify ultralytics installation and check GPU availability
import ultralytics
ultralytics.checks()

# Import Roboflow for dataset download
from roboflow import Roboflow

# Get API key from environment variable
roboflow_api_key = os.getenv("ROBOFLOW_API_KEY")
if not roboflow_api_key:
    raise ValueError("ROBOFLOW_API_KEY not found in environment variables. Please set it in .env file.")

# Download the paraglider dataset from Roboflow (version 8)
rf = Roboflow(api_key=roboflow_api_key)
project = rf.workspace("initialtrial").project("paraglider_recognition")
dataset = project.version(8).download("yolov8")



Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.3/112.6 GB disk)
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to paraglider_recognition-8 in yolov8:: 100%|██████████| 214/214 [00:00<00:00, 620.40it/s]


In [ ]:
## Step 1: Environment Setup
Dataset has been downloaded from Roboflow. Now setting up the working directory and loading required libraries.


In [ ]:
# Set working directory and display the home path
import os
HOME = os.getcwd()
print(f"Working directory: {HOME}")


/content


In [ ]:
# Import utilities for model training and result visualization
from ultralytics import YOLO
from IPython.display import display, Image


In [ ]:
# Verify PyTorch installation and check GPU availability
import torch
device_info = torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'
print('Setup complete. Using torch %s %s' % (torch.__version__, device_info))


Setup complete. Using torch 2.8.0+cu126 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15095MB, multi_processor_count=40, uuid=97dccfc4-79ae-5a84-9506-c2e465a0344c, pci_bus_id=0, pci_device_id=4, pci_domain_id=0, L2_cache_size=4MB)


In [ ]:
# Create datasets directory and download the paraglider training dataset
import os
from dotenv import load_dotenv

load_dotenv()

# Create datasets folder in home directory
!mkdir -p {HOME}/datasets
%cd {HOME}/datasets

# Get API key from environment variable and download dataset
roboflow_api_key = os.getenv("ROBOFLOW_API_KEY")
if not roboflow_api_key:
    raise ValueError("ROBOFLOW_API_KEY not found in environment variables. Please set it in .env file.")

rf = Roboflow(api_key=roboflow_api_key)
project = rf.workspace("initialtrial").project("paraglider_recognition")
dataset = project.version(8).download("yolov8")



mkdir: cannot create directory ‘/content/datasets’: File exists
/content/datasets
loading Roboflow workspace...
loading Roboflow project...


In [ ]:
# Display the dataset location path
dataset.location


'/content/datasets/paraglider_recognition-8'

In [ ]:
# Train YOLOv8 model on paraglider dataset
# Returns to home directory and starts training
%cd {HOME}

# Training parameters:
# - task: detect (object detection)
# - mode: train (training mode)
# - model: yolov8.pt (YOLOv8 pretrained model)
# - data: path to data.yaml configuration
# - epochs: 25 (number of training epochs)
# - imgsz: 800 (image size)
# - plots: True (generate training plots)
!yolo task=detect mode=train model=yolov8.pt data={dataset.location}/data.yaml epochs=25 imgsz=800 plots=True


/content
Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
             ^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/cfg/__init__.py", line 955, in entrypoint
    model = YOLO(model, task=task)
            ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/models/yolo/model.py", line 81, in __init__
    super().__init__(model=model, task=task, verbose=verbose)
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/engine/model.py", line 149, in __init__
    self._load(model, task=task)
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/engine/model.py", line 288, in _load
    self.model, self.ckpt = load_checkpoint(weights)
                            ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/nn/tasks.py", line 1461, in load_checkpoint
    ckpt, weight = torch_safe_load(weight)  # load ckpt
                   

In [ ]:
# Import libraries for visualization
import glob
from IPython.display import Image, display


In [ ]:
# Display first 3 prediction results from training
for image_path in glob.glob(f'{HOME}/runs/detect/predict3/*.jpg')[:3]:
      display(Image(filename=image_path, width=600))
      print("\n")


In [ ]:
# Display current working directory
HOME


'/content'

In [ ]:
# Deploy the trained model back to Roboflow for versioning and management
project.version(dataset.version).deploy(model_type="yolov8", model_path=f"{HOME}/runs/detect/train/")


Dependency ultralytics==8.0.196 is required but found version=8.3.229, to fix: `pip install ultralytics==8.0.196`
Would you like to continue with the wrong version of ultralytics? y/n: y
An error occured when getting the model upload URL: This version already has a trained model. Please generate and train a new version in order to upload model to Roboflow.


In [ ]:
# Install QAI Hub models for edge device optimization
!pip install qai-hub-models[yolov8s]


In [ ]:
# Configure QAI Hub with API token for model optimization
import os
from dotenv import load_dotenv

load_dotenv()

# Get QAI Hub API token from environment variable
qai_hub_token = os.getenv("QAI_HUB_API_TOKEN")
if not qai_hub_token:
    raise ValueError("QAI_HUB_API_TOKEN not found in environment variables. Please set it in .env file.")

# Configure QAI Hub with the token
!qai-hub configure --api_token {qai_hub_token}



2025-11-20 22:00:53.982 - INFO - Enabling verbose logging.
qai-hub configuration saved to /root/.qai_hub/client.ini
==================== /root/.qai_hub/client.ini ====================
[api]
api_token = cqqq19zmvy91nm4ozogusg1dsca352gbkfwxz1ki
api_url = https://app.aihub.qualcomm.com
web_url = https://app.aihub.qualcomm.com
verbose = True




In [ ]:
# Retrieve and store QAI Hub API token for model export
import os
from dotenv import load_dotenv

load_dotenv()

# Get QAI Hub API token from environment variable for use in export
qc_key = os.getenv("QAI_HUB_API_TOKEN")
if not qc_key:
    raise ValueError("QAI_HUB_API_TOKEN not found in environment variables. Please set it in .env file.")



In [ ]:
# Export YOLOv8 model for Qualcomm Snapdragon (QCS6490) with quantization
# Parameters:
# - quantize w8a8: 8-bit weight and activation quantization
# - device: QCS6490 (Qualcomm Snapdragon platform)
# - target-runtime: tflite (TensorFlow Lite format)
!yes | python -m qai_hub_models.models.yolov8_det.export --quantize w8a8 --device="QCS6490" --ckpt-name /content/runs/detect/train/weights/best.pt --device-os linux --target-runtime tflite --output-dir .



E0000 00:00:1763676507.354591   13078 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763676507.364279   13078 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763676507.387737   13078 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763676507.387771   13078 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763676507.387778   13078 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763676507.387785   13078 computation_placer.cc:177] computation placer already registered. Please check linka

## Training Complete!

The model has been:
1. ✅ Trained on the paraglider dataset for 25 epochs
2. ✅ Deployed to Roboflow for versioning
3. ✅ Exported for edge devices (QCS6490)

The trained model is available in `{HOME}/runs/detect/train/weights/best.pt` and can be used for inference on new images.
